In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install psycopg2-binary
!pip install sqlalchemy

In [2]:
import psycopg2
import pandas as pd
import math
from os import environ
from time import time
import numpy as np
import json
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from matplotlib.dates import DateFormatter
import datetime
import pytz

In [3]:
SECONDS_IN_HOUR = 3600

In [4]:
# Settings for plots
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

In [5]:
def get_relevant_periods(period_start, period_end):

    lower_bound = period_start - (period_start % SECONDS_IN_HOUR)
    upper_bound = period_end - (period_end % SECONDS_IN_HOUR) + SECONDS_IN_HOUR

    amount_of_full_hours = int((upper_bound - lower_bound) / SECONDS_IN_HOUR)

    ranges = []
    for i in range(amount_of_full_hours):
        ranges.append(f"{lower_bound + i * SECONDS_IN_HOUR}-{lower_bound + (i + 1) * SECONDS_IN_HOUR}")

    return ranges

def get_time_running_in_hours(row):

    # The device was running for the full period
    if row['event_timestamp'] < row['period_start'] and row['event_timestamp_next'] > row['period_end']:
        return SECONDS_IN_HOUR / SECONDS_IN_HOUR

    # The device started and stopped withing this period
    if row['event_timestamp'] > row['period_start'] and row['event_timestamp_next'] < row['period_end']:
        return (row['event_timestamp_next'] - row['event_timestamp']) / SECONDS_IN_HOUR

    # The device started in this period and was still running at the end of this period
    if row['event_timestamp'] > row['period_start'] and row['event_timestamp_next'] > row['period_end']:
        return (row['period_end'] - row['event_timestamp']) / SECONDS_IN_HOUR

    # The device started before this period, but stopped running in this period
    if row['event_timestamp'] < row['period_start'] and row['event_timestamp_next'] < row['period_end']:
        return (row['event_timestamp_next'] - row['period_start']) / SECONDS_IN_HOUR

In [6]:
# Create an engine instance
engine   = create_engine(f"postgresql+psycopg2://{environ['DB_USERNAME']}:{environ['DB_PASSWORD']}@{environ['DB_HOSTNAME']}:5432/homeconnect", pool_recycle=3600);
with engine.connect() as conn:
    df = pd.read_sql("select * from operation_states", conn);

# Create more descriptive column names
df = df.rename(columns={'datetime_stored': 'event_timestamp'})

df['state'] = df.apply(lambda row: row['state'].replace('BSH.Common.EnumType.OperationState.',''), axis=1)
df['event_timestamp_next'] = df['event_timestamp'].shift(-1, fill_value=int(time()))
df['state_next'] = df['state'].shift(-1)
df = df[df['state'] == 'Run']

df['periods'] = df.apply(lambda row: get_relevant_periods(row['event_timestamp'], row['event_timestamp_next']), axis=1)
df = df.explode('periods')

df['period_start'] = df.apply(lambda row: int(row['periods'].split('-')[0]), axis=1)
df['period_end'] = df.apply(lambda row: int(row['periods'].split('-')[1]), axis=1)

df['time_running_in_hours'] = df.apply(lambda row: get_time_running_in_hours(row), axis=1)
df['period_start'] = pd.to_datetime(df['period_start'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df['period_end'] = pd.to_datetime(df['period_end'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))

df = df.drop(['event_timestamp', 'state', 'event_timestamp_next', 'state_next', 'periods', 'id'], axis=1)

df = df.groupby(['appliance_name', 'period_start', 'period_end']).sum(['time_running_in_hours']).reset_index()

df

,appliance_name,period_start,period_end,time_running_in_hours
0,dishwasher,2023-10-16 00:00:00+02:00,2023-10-16 01:00:00+02:00,0.398333
1,dishwasher,2023-10-16 01:00:00+02:00,2023-10-16 02:00:00+02:00,1.000000
2,dishwasher,2023-10-16 02:00:00+02:00,2023-10-16 03:00:00+02:00,1.000000
3,dishwasher,2023-10-16 03:00:00+02:00,2023-10-16 04:00:00+02:00,1.000000
4,dishwasher,2023-10-16 04:00:00+02:00,2023-10-16 05:00:00+02:00,1.000000
5,dishwasher,2023-10-16 05:00:00+02:00,2023-10-16 06:00:00+02:00,0.448889
6,dishwasher,2023-10-17 09:00:00+02:00,2023-10-17 10:00:00+02:00,0.331389
7,dishwasher,2023-10-17 10:00:00+02:00,2023-10-17 11:00:00+02:00,1.000000
8,dishwasher,2023-10-17 11:00:00+02:00,2023-10-17 12:00:00+02:00,1.000000
9,dishwasher,2023-10-17 12:00:00+02:00,2023-10-17 13:00:00+02:00,1.000000


In [120]:
# Read sqlite query results into a pandas DataFrame
def get_latest_data() -> pd.DataFrame:
    con = sqlite3.connect("minimal_footprint/integrations/nibe/nibe.db")
    df = pd.read_sql_query("SELECT * from data WHERE datetime_stored > STRFTIME('%s') - (36 * 60 * 60)", con)

    df['datetime_stored'] = pd.to_datetime(df['datetime_stored'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))

    return df

# Sanitize different types of values for the dataframe
def sanitize_values(row):

    try:
        return float(str(row['display_value']).replace(row['unit'], ""))
    except ValueError:
        try:
            return mappings_for_parameter_ids[row['parameter_id']][row['display_value']]
        except KeyError:
            print(f"Value {row['display_value']} for {row['parameter_id']} is missing.")

    return row['display_value']

In [121]:
# Load the parameter dictionary
f = open("minimal_footprint/integrations/nibe/F1255PC.json")
parameter_dict = json.load(f)
f.close()

# Get the column names for parameter ids and mappings for parameter ids
column_names_for_parameter_ids = {parameter['parameter_id']: parameter['column_header'] for parameter in parameter_dict['parameters_active']}
mappings_for_parameter_ids = {parameter['parameter_id']: parameter['mappings'] for parameter in parameter_dict['parameters_active']}

In [27]:
# Retrieve the latest data
df = get_latest_data()

# Add the proper column headers
df['column_header'] = df.apply(lambda row: column_names_for_parameter_ids[row['parameter_id']], axis=1)

# Create 'values' from 'display values'
df['value'] = df.apply(lambda row: sanitize_values(row), axis=1)

# Pivot
df_plot = df.pivot_table(index=['datetime_stored'], columns=['column_header'], values='value').fillna(np.NaN)

df_plot

# Add operational statuses
COMPRESSOR_STOPPED_STATUS = 20
df_plot['compressor_is_running'] = df_plot.apply(lambda row: row['compressor_state'] != COMPRESSOR_STOPPED_STATUS, axis=1)
df_plot['heat_flow_pump_is_running'] = df_plot.apply(lambda row: row['heat_medium_flow_pump_speed'] > 0, axis=1)
df_plot['brine_flow_pump_is_running'] = df_plot.apply(lambda row: row['brine_flow_pump_speed'] > 0, axis=1)
df_plot['heat_distributed_to_house'] = df_plot.apply(lambda row: row['heat_medium_flow_temperature'] - row['heat_medium_flow_return_temperature'], axis=1)
df_plot['heat_obtained_in_source'] = df_plot.apply(lambda row: row['temperature_brine_in'] - row['temperature_brine_out'], axis=1)

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(df_plot.index, 0, 1, where=df_plot['compressor_is_running'], alpha=0.4, transform=ax.get_xaxis_transform())

# Define the date format
date_form = DateFormatter("%d-%m %H:%M", tz="Europe/Amsterdam")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_plot.index, df_plot['spa_price_level'], label="Smart Price Adaption price level.", alpha=0.2)
plt.plot(df_plot.index, df_plot['electricity_price'], label="Electricity price.", color="blue")
plt.plot(df_plot.index, df_plot['heat_flow_pump_is_running'], label="Heat flow pump is running.", color="red")
plt.plot(df_plot.index, df_plot['brine_flow_pump_is_running'], label="Brine flow pump is running.")

ax.legend()
plt.title("Pump activity and electricity prices during compressor activity.")

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(df_plot.index, 0, 1, where=df_plot['heat_flow_pump_is_running'], alpha=0.4, transform=ax.get_xaxis_transform())

# Define the date format
date_form = DateFormatter("%d-%m %H:%M", tz="Europe/Amsterdam")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_plot.index, df_plot['hot_water_temperature_bottom'], label="Hot water temperature bottom.", color="red")
plt.plot(df_plot.index, df_plot['spa_hot_water_comfort_mode'], label="Smart Price Adaption Hot Water Comfort Mode.")
plt.plot(df_plot.index, df_plot['stop_temperature_hot_water_normal'], label="Stop temperature hot water normal.")
plt.plot(df_plot.index, df_plot['heat_distributed_to_house'], label="Heat distributed to house.")
plt.plot(df_plot.index, df_plot['heat_obtained_in_source'], label="Heat obtained in source.")

ax.grid(axis='y')
ax.legend()
plt.title("Heat flow and brine flow activity during heat flow pump activity.")

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(df_plot.index, 0, 1, where=df_plot['brine_flow_pump_is_running'], alpha=0.4, transform=ax.get_xaxis_transform())

# Define the date format
date_form = DateFormatter("%d-%m %H:%M")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_plot.index, df_plot['outdoor_temperature_avg'], label="Average outdoor temperature.", color="blue")
plt.plot(df_plot.index, df_plot['heat_medium_flow_temperature'], label="Heat medium flow temperature.")
plt.plot(df_plot.index, df_plot['heat_medium_flow_calculated_temperature'], label="Calculated heat medium flow temperature.")
plt.plot(df_plot.index, df_plot['heat_medium_flow_return_temperature'], label="Heat medium flow return temperature.", color="red")
plt.plot(df_plot.index, df_plot['heat_distributed_to_house'], label="Heat distributed to house.", color="green")

ax.grid(axis='y')
ax.legend()
plt.title("Heat flow temperatures (in and out) during heat flow pump activity.")

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(df_plot.index, 0, 1, where=df_plot['brine_flow_pump_is_running'], alpha=0.4, transform=ax.get_xaxis_transform())

# Define the date format
date_form = DateFormatter("%d-%m %H:%M", tz="Europe/Amsterdam")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_plot.index, df_plot['temperature_brine_in'], label="Temperature brine in.")
plt.plot(df_plot.index, df_plot['temperature_brine_out'], label="Temperature brine out.", color="red")
plt.plot(df_plot.index, df_plot['heat_obtained_in_source'], label="Heat obtained in source.", color="green")

ax.grid(axis='y')
ax.legend()
plt.title("Brine in and out temperatures during brine flow pump activity.")

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(df_plot.index, 0, 1, where=df_plot['compressor_is_running'], alpha=0.4, transform=ax.get_xaxis_transform())

# Define the date format
date_form = DateFormatter("%d-%m %H:%M")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_plot.index, df_plot['temperature_hot_gas'], label="Temperature hot gas (BT14).", color="red")
plt.plot(df_plot.index, df_plot['temperature_suction_gas'], label="Temperature suction gas (BT17).")
plt.plot(df_plot.index, df_plot['temperature_liquid_line'], label="Temperature liquid line (BT15).")

ax.grid(axis='y')
ax.legend()
plt.title("Refrigerant flow temperatures during compressor activity.")

plt.show()

In [ ]:
con = sqlite3.connect("minimal_footprint/integrations/tibber/tibber.db")
df_tibber_consumption = pd.read_sql_query("SELECT * from consumption ORDER BY period_start", con)

df_tibber_consumption['period_start'] = pd.to_datetime(df_tibber_consumption['period_start'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df_tibber_consumption['period_end'] = pd.to_datetime(df_tibber_consumption['period_end'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df_tibber_consumption = df_tibber_consumption.fillna(0)

df_tibber_consumption.tail()

In [ ]:
con = sqlite3.connect("minimal_footprint/integrations/tibber/tibber.db")
df_tibber_production = pd.read_sql_query("SELECT * from production ORDER BY period_start", con)

df_tibber_production['period_start'] = pd.to_datetime(df_tibber_production['period_start'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df_tibber_production['period_end'] = pd.to_datetime(df_tibber_production['period_end'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df_tibber_production = df_tibber_production.fillna(0)

df_tibber_production.tail()

In [ ]:
con = sqlite3.connect("minimal_footprint/integrations/enphase/enphase.db")
df_enphase = pd.read_sql_query("SELECT * from production ORDER BY period_start", con)

df_enphase['period_start'] = pd.to_datetime(df_enphase['period_start'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df_enphase['period_end'] = pd.to_datetime(df_enphase['period_end'],unit='s', utc=True).dt.floor('S').map(lambda x: x.tz_convert('Europe/Amsterdam'))
df_enphase = df_enphase.fillna(0)

df_enphase.tail()

In [36]:
import functools as ft

dfs = [df_tibber_consumption, df_tibber_production, df_enphase]
df_final = ft.reduce(lambda left, right: pd.merge(left, right, how='inner', on=['period_start', 'period_end']), dfs)
df_final = df_final.rename(columns={"profit": "revenue", "unit_price_x": "price_per_kwh", "consumption": "kwh_purchased",  "production": "kwh_sold", "watt_hours": "watts_generated"})

df_final['kwh_generated'] = df_final.apply(lambda row: row['watts_generated'] / 1000 , axis=1)
df_final['kwh_used'] = df_final.apply(lambda row: row['kwh_generated'] - row['kwh_sold'] + row['kwh_purchased'] , axis=1)
df_final['kwh_used_generated_power'] = df_final.apply(lambda row: row['kwh_used'] - row['kwh_purchased'], axis=1)
df_final['kwh_used_purchased_power'] = df_final['kwh_purchased']
df_final['profit'] = df_final.apply(lambda row: row['revenue'] - row['cost'] , axis=1)

df_final = df_final.drop(['unit_price_y', 'consumption_unit', 'production_unit', 'watts_generated', 'id', 'id_x', 'id_y'], axis=1)
df_final = df_final.fillna(0)

# # Add total profits and costs
df_final['total_revenue'] = df_final['revenue'].cumsum()
df_final['total_costs'] = df_final['cost'].cumsum()
df_final['total_profit'] = df_final['profit'].cumsum()

pd.set_option('display.max_rows', None)

In [ ]:
fig, ax = plt.subplots()

# Define the date format
date_form = DateFormatter("%d-%m %H:%M", tz="Europe/Amsterdam")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_final['period_end'], df_final['kwh_used'], label="Energy used in the house.", color="black", alpha=0.0, marker=".")
plt.plot(df_final['period_end'], df_final['kwh_used_purchased_power'], label="Used energy that was purchased", alpha=0.0, color="red", marker="2")
ax.fill_between(df_final['period_end'], df_final['kwh_used_purchased_power'], alpha=1.0, color="red", interpolate=True)
ax.fill_between(df_final['period_end'], df_final['kwh_used_purchased_power'], df_final['kwh_used'], alpha=1.0, color="green", interpolate=True)
plt.plot(df_final['period_end'], df_final['kwh_generated'], label="Generated power.", alpha=0.0, color="gold", marker="4")
ax.fill_between(df_final['period_end'], df_final['kwh_used'], df_final['kwh_generated'], where=(df_final['kwh_used'] < df_final['kwh_generated']), alpha=1.0, color="gold", interpolate=True)
plt.plot(df_final['period_end'], df_final['price_per_kwh'], label="Price per KWH.", alpha=0.5, color="black", marker="1")
plt.plot(df_final['period_end'], df_final['total_profit'], label="Total profit.", alpha=0.5, color="silver", marker="x")


plt.title("Energy interactions with the grid.")
ax.grid(axis='y')
ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots()

# Define the date format
date_form = DateFormatter("%d-%m %H:%M", tz="Europe/Amsterdam")
ax.xaxis.set_major_formatter(date_form)

plt.plot(df_final['period_end'], df_final['profit'], label="Profit per hour (in euros).", color="black", drawstyle="steps-pre")
plt.plot(df_final['period_end'], df_final['total_profit'], label="Total profit (in euros).", color="gold", drawstyle="steps-pre")

ax.grid(axis='y')
ax.legend()
plt.title("Profit.")

plt.show()